## Data Cleaning

Before we can start with our analysis, we have to check and prepare our data. This is a crucial task and is necessary to prevent data-driven mistakes.

In [1]:
# Basic Python Libs
import json
import numpy as np
import pandas as pd

First off, we should get to know the data. So let's start with some simple questions.

* How many orders were recorded?
* Which features were considered?
* Are all orders labeled as kept/returned?
* Do we have categorial and/or numerical features?
* Does every features has a value for every order?
* Are the values for the specific features consistent?
* ... ?

Always keep asking yourself questions like that. You can never run enough checks to make sure you have a clean dataset to work with! So let's dig into the data and see how we can somewhat efficiently go about these questions. 

In [4]:
data_path = "Data\\"
file_name = "orders.csv"

df = pd.read_csv(data_path+file_name, delimiter=',')
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
order,78834,NaN,NaN,NaN,39416.5,22757.6,0,19708.2,39416.5,59124.8,78833
age_in_years,78834,NaN,NaN,NaN,35.8873,10.3745,15,28,35,42,79
category,78834,2,T-Shirts,41135,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_group,78834,8,Slim fit,20504,NaN,NaN,NaN,NaN,NaN,NaN,NaN
height_in_cm,78803,NaN,NaN,NaN,181.033,6.7853,149.4,176.3,180.7,185.3,219.5
date_shipped,78834,253,2017-06-28 00:00:00,758,NaN,NaN,NaN,NaN,NaN,NaN,NaN
size,78834,5,M,26691,NaN,NaN,NaN,NaN,NaN,NaN,NaN
color,78834,17,Blue,40238,NaN,NaN,NaN,NaN,NaN,NaN,NaN
item_returned,78834,NaN,NaN,NaN,0.506241,0.499964,0,0,1,1,1
weight_in_kg,78797,NaN,NaN,NaN,83.0411,11.5758,48.3,75.1,82.4,89.9,179.8


In [42]:
print(df.dtypes)

order              int64
age_in_years       int64
category          object
product_group     object
height_in_cm     float64
date_shipped      object
size              object
color             object
item_returned      int64
weight_in_kg     float64
price_in_euro    float64
dtype: object


Our starting point is the describe feature for a Pandas dataframe.

Here we have a summary of our features.

Numerical:
* order
* age_in_years
* height_in_cm
* item_returned
* weight_in_kg
* price_in_euro

Categorial:
* category
* product_group
* size
* color

Time:
* date_shipped

Excellent choice for the names of the features, since it's clear what was recorded.  
We can also see that we have __$78834$ orders__ in our data.  
For all features but __height_in_cm__ and __weight_in_kg__, we have a value for all orders. Seems like some people were shy about their height and/or weight. Hence we should also see a label for all orders.  
Furthermore we have both numerical and categorial features plus one date feature. We can also graps that the features __age_in_years__, __height_in_cm__, __weight_in_kg__ belong to the __customer__ with the other features belonging to the product/shipping.  
As for our categorial features, we can see that there seems to be $2$ __categories__, $8$ __products__, $5$ __sizes__ and $17$ __colors__. Additionally the __most frequent value__ for these features are given.  
Lastly we already get a preview about the __details__ of our numerical features like the __mean age, height and weight of our customer, the return ratio and the average price paid per order.__   
But at the moment, we are not interested into exploring our data in detail. We just want to understand what we have to work with and make sanity checks.

We will start with the __missing values in __height_in_cm__ and __weight_in_kg__. There are several options on how to deal with missing values. At most 68 orders are affected, which corresponds to less than $0.1\,\%$ of our data. So removing the data is certainly a good option. 

* Comment: An other useful way to check for missing vallues
```python
df.isnull().sum()
```

In [12]:
df = df.dropna()
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
order,78797,NaN,NaN,NaN,39418.6,22757.8,0,19711,39422,59126,78833
age_in_years,78797,NaN,NaN,NaN,35.8898,10.3755,15,28,35,42,79
category,78797,2,T-Shirts,41108,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_group,78797,8,Slim fit,20499,NaN,NaN,NaN,NaN,NaN,NaN,NaN
height_in_cm,78797,NaN,NaN,NaN,181.033,6.78546,149.4,176.3,180.7,185.3,219.5
date_shipped,78797,253,2017-06-28 00:00:00,758,NaN,NaN,NaN,NaN,NaN,NaN,NaN
size,78797,5,M,26689,NaN,NaN,NaN,NaN,NaN,NaN,NaN
color,78797,17,Blue,40218,NaN,NaN,NaN,NaN,NaN,NaN,NaN
item_returned,78797,NaN,NaN,NaN,0.506123,0.499966,0,0,1,1,1
weight_in_kg,78797,NaN,NaN,NaN,83.0411,11.5758,48.3,75.1,82.4,89.9,179.8


We got rid of our missing values and are left with $78797$ orders. Next up we should explore the values of our features, starting with the categorial ones.

In [13]:
print(np.unique(df.category))
print(np.unique(df.product_group))
print(np.unique(df.color))
print(np.unique(df['size'])) # bad naming! df.size is a command"

['Shirts casual' 'T-Shirts']
['Polo shirt longsleeves' 'Polo shirt shortsleeves' 'Regular fit'
 'Slim fit' 'T-shirt Basic' 'T-shirt Print' 'T-shirt long sleeves'
 'T-shirt striped / patterned']
['Beige' 'Black' 'Blue' 'Bluec' 'Brown' 'Green' 'Grey' 'Grey  ' 'Metal'
 'Multicolor' 'Orange' 'Pink' 'Red' 'Turquoise' 'Violet' 'White' 'Yellow']
['L' 'M' 'S' 'XL' 'XXL']


The $2$ __categories__ are __Shirts casual__, __T-Shirts__. They are divded into $8$ groups from __Polo shirt longsleeves__ to __T-shirt striped / patterned__. There seems to be no typos here and the the values seem to be fitting.

However, If we take a look at the __colors__, we can spot the colors __Blue__ and __Bluec__ as well as __Grey__ and __Grey(space)__. Seems like something went wrong in the data collection. We should investigate how this happened and determine if this is a simple typo or an other error. Depending on the answer we should correct the typo or dismiss the affected orders. Luckly it is just a typo! Bluec should be Blue and Grey(space) should be Grey. We can easily fix this. The remaining color values seem fine.  

Lastly we have the $5$ known __sizes__ from __S__ up to __XXL__. Nothing to worry about.

In [15]:
df.color[df['color'] == 'Grey  '] = 'Grey'
df.color[df['color'] == 'Bluec'] = 'Blue'

c:\users\oli\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
c:\users\oli\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Now we can take a look at our numerical values. We should check if the ranges make sense and if there are any outliners.

In [25]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order,78797.0,39418.607130,22757.840711,0.00,19711.0,39422.00,59126.00,78833.00
age_in_years,78797.0,35.889844,10.375490,15.00,28.0,35.00,42.00,79.00
height_in_cm,78797.0,181.032505,6.785464,149.40,176.3,180.70,185.30,219.50
item_returned,78797.0,0.506123,0.499966,0.00,0.0,1.00,1.00,1.00
weight_in_kg,78797.0,83.041101,11.575792,48.30,75.1,82.40,89.90,179.80
price_in_euro,78797.0,7.460359,1.363198,2.24,6.3,7.39,8.52,11.66


The labeling for the returned order is given by the integers $0$ and $1$, with the later marking the returned orders.  
As for the order feature, it seems to be a simple list. Hence this feature won't be useful for us.   
The __age_in_years__ are just integers from $15$ to $79$. So we have both very young and also older customers!

The features __height_in_cm__, __weight_in_kg__ and __price_in_euro__ are all floats. For both height and weight we have $1$ decimal, while we have $2$ decimals for the price. For height, the range of the values makes sense. We have short and tall customers. The range for the weight is pretty wide with a minimum of less than $50\,$kg and a maximum of almost $180\,$kg. Nothing humanly impossible, but we should take a deeper look here. A usefull feature would be a the BMI.

As for the price, we have a bargain bottom of just $2.24\,$€ and a maximum of just $11.66\,$€. Though, most prices are in the range of $6\,$€ to $8.50\,$€. We may have some outliners, which could potentially be typos.

In [27]:
df.to_csv(r'order_cleaned.csv')